Stats on Ca2+ imaging with miniscope and Vigilance States

In [1]:
import statsmodels.api as sm
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import pickle
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf

%matplotlib widget


In [11]:
# Specify the directory containing the Excel files
directory = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording/"

# Initialize an empty list to store the dataframes
dfs = []
df=[]

NrSubtype='All' # to changed 

if NrSubtype=='L1':
    nametofind='VigilanceState_GlobalResults'
else:
    nametofind='VigilanceState_AllNr_GlobalResults'#for Purple, ThreeColDots and ThreeBlueLines

# Recursively traverse the directory structure
for root, _, files in os.walk(directory):
    for filename in files:
        # Check if the file is an Excel file and contains the specified name
        if filename.endswith('.xlsx') and nametofind in filename: 
            # Construct the full path to the file
            filepath = os.path.join(root, filename)
            # Read the Excel file into a dataframe and append it to the list
            df = pd.read_excel(filepath, index_col=0)      
            dfs.append(df)

# Concatenate all dataframes into a single dataframe
combined_df = pd.concat(dfs, ignore_index=True)

In [12]:
#Remove non defined Unique Units 
combined_df = combined_df[combined_df['Unique_Unit'] != '[]']
combined_df['Unique_Unit'] = combined_df['Unique_Unit'].astype(str)
combined_df['UnitNumber'] = combined_df['UnitNumber'].astype(str)
combined_df['UnitValue'] = combined_df['UnitValue'].astype(str)

In [13]:
combined_df['NormalizedAUC_calcium'] = combined_df['AUC_calcium'] / combined_df['DurationSubstate']
combined_df['NormalizedAUC_spike'] = combined_df['AUC_spike'] + combined_df['DurationSubstate']

combined_df['Unit_ID'] = combined_df['Mice'] + combined_df['Unique_Unit']
combined_df['Substate_ID'] = combined_df['Mice'] + combined_df['Session'] + combined_df['Substate'] + combined_df['SubstateNumber'].astype(str)

In [14]:
combined_df

,Mice,Session,Session_Time,Unique_Unit,UnitNumber,UnitValue,Substate,SubstateNumber,DurationSubstate,CalciumActivity,...,Avg_CalciumActivity,Avg_AUC_calcium,SpikeActivity,AUC_spike,Avg_SpikeActivity,Avg_AUC_spike,NormalizedAUC_calcium,NormalizedAUC_spike,Unit_ID,Substate_ID
0,Purple,session11,NaN,4,0,0,NREM,0,50.850000,2.052812,...,0.978553,20549.617897,0.009943,10.101899,0.003531,74.153024,40.990442,60.951899,Purple4,Purplesession11NREM0
1,Purple,session11,NaN,4,0,0,Wake,1,5.000000,2.512997,...,0.978553,20549.617897,0.011343,1.122932,0.003531,74.153024,49.220511,6.122932,Purple4,Purplesession11Wake1
2,Purple,session11,NaN,4,0,0,NREM,2,20.000000,2.510358,...,0.978553,20549.617897,0.008442,3.368354,0.003531,74.153024,49.948488,23.368354,Purple4,Purplesession11NREM2
3,Purple,session11,NaN,4,0,0,Wake,3,5.000000,2.673079,...,0.978553,20549.617897,0.011059,1.034895,0.003531,74.153024,52.406708,6.034895,Purple4,Purplesession11Wake3
4,Purple,session11,NaN,4,0,0,NREM,4,15.000000,2.821749,...,0.978553,20549.617897,0.007733,2.263913,0.003531,74.153024,56.083354,17.263913,Purple4,Purplesession11NREM4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17043,ThreeColDotsOK,session34,NaN,332,7,86,NREM,26,30.000000,0.168276,...,0.350983,8774.569221,0.002348,2.111042,0.003795,94.886472,5.028862,32.111042,ThreeColDotsOK332,ThreeColDotsOKsession34NREM26
17044,ThreeColDotsOK,session34,NaN,332,7,86,N2,27,30.000000,0.792832,...,0.350983,8774.569221,0.008506,7.647209,0.003795,94.886472,23.738069,37.647209,ThreeColDotsOK332,ThreeColDotsOKsession34N227
17045,ThreeColDotsOK,session34,NaN,332,7,86,NREM,28,15.000000,0.117450,...,0.350983,8774.569221,0.000000,0.000000,0.003795,94.886472,3.496375,15.000000,ThreeColDotsOK332,ThreeColDotsOKsession34NREM28
17046,ThreeColDotsOK,session34,NaN,332,7,86,REM,29,45.000000,0.876555,...,0.350983,8774.569221,0.009793,13.211010,0.003795,94.886472,26.272307,58.211010,ThreeColDotsOK332,ThreeColDotsOKsession34REM29


Save big dataset for stats

In [15]:
filenameOut = f'{directory}{NrSubtype}_VigilanceStates_GrandGlobalAB.xlsx'
writer = pd.ExcelWriter(filenameOut)
combined_df.to_excel(writer)
writer.close()

Do Stats

From Formation Python CRNL

In [16]:
"""md = smf.mixedlm('AUC_calcium ~ Substate', combined_df, groups=combined_df['Unique_Unit']) 
## mixed lm: idem ols (avec formula) mais cette fois on rajoute une variable groups, qui indique le facteur de fixed effect
mdf = md.fit()
print(mdf.summary())"""

"md = smf.mixedlm('AUC_calcium ~ Substate', combined_df, groups=combined_df['Unique_Unit']) \n## mixed lm: idem ols (avec formula) mais cette fois on rajoute une variable groups, qui indique le facteur de fixed effect\nmdf = md.fit()\nprint(mdf.summary())"

From Chat GPT

In [17]:
"""AttributeError# Fit the mixed-effects model
mixedlm_model = sm.MixedLM.from_formula('AUC_spike ~ Substate', groups='Unique_Unit', data=combined_df)
result = mixedlm_model.fit()

# Print model summary
print(result.summary())"""

"AttributeError# Fit the mixed-effects model\nmixedlm_model = sm.MixedLM.from_formula('AUC_spike ~ Substate', groups='Unique_Unit', data=combined_df)\nresult = mixedlm_model.fit()\n\n# Print model summary\nprint(result.summary())"

In [18]:
"""# Get the coefficients from the model results
coefficients = result.params

# Plot the coefficients
plt.bar(coefficients.index, coefficients)
plt.xlabel('Coefficient')
plt.ylabel('Value')
plt.title('Coefficients of Mixed Effects Model')
plt.show()"""

"# Get the coefficients from the model results\ncoefficients = result.params\n\n# Plot the coefficients\nplt.bar(coefficients.index, coefficients)\nplt.xlabel('Coefficient')\nplt.ylabel('Value')\nplt.title('Coefficients of Mixed Effects Model')\nplt.show()"

From forum

In [19]:
"""model = smf.mixedlm(formula='AUC_spike ~ Substate',
#                re_formula='~ Unique_Unit',
                data=combined_df, groups='Unique_Unit').fit()
print(model.summary())"""

"model = smf.mixedlm(formula='AUC_spike ~ Substate',\n#                re_formula='~ Unique_Unit',\n                data=combined_df, groups='Unique_Unit').fit()\nprint(model.summary())"